In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM
from keras.layers import Dropout
from os.path import isfile, join
from keras.layers import Flatten, Conv1D, Input, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras import layers as L
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
import pickle
import os
from os import listdir
import pywt
from pywt import dwt, wavedec
from os.path import isfile, join
import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2021-12-05 12:02:35.674480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-05 12:02:35.674538: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df,
                 batch_size, validation,
                 shuffle=True):

        self.df = df.copy()
        if validation:
            self.df = self.df[25000:]
        else:
            self.df = self.df[:25000]
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path):
    
        objects = []
        with (open(path, "rb")) as openfile:
            while True:
                try:
                    objects.append(pickle.load(openfile))
                except EOFError:
                    break
        return objects[0]["audio"]

    
    def __get_output(self, tag):
        return lab[tag]
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        files = batches["file"]
        tag = batches["tag"]

        X_batch = np.asarray([self.__get_input(x) for x in files])

        y_batch = np.asarray([self.__get_output(y) for y in tag])

        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
dft = pd.read_csv('dataGencoif3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelcoif3.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [3]:
dft = pd.read_csv('dataGencoif6.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100

def AttRNNSpeechModel(rnn_func=L.LSTM):
    inputs = Input(shape=(7, 20, 216))
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(8, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)
    x = L.Bidirectional(rnn_func(32, return_sequences=True)
                        )(x)

    xFirst = L.Lambda(lambda q: q[:, -1])(x)
    query = L.Dense(64)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modelcoif6.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

2021-12-05 12:02:41.491787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-05 12:02:41.492210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-05 12:02:41.492567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-05 12:02:41.492904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libc

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7, 20, 216)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 7, 20, 128)   138368      ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 7, 20, 128)  512         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

/tmp/ipykernel_11244/948521708.py:76: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)
2021-12-05 12:02:46.465653: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 2100 num_cores: 64 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 1048576 l3_cache_size: 23068672 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


781/781 [==============================] - ETA: 0s - loss: 2.5886 - accuracy: 0.3345

2021-12-05 12:04:15.941109: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 2100 num_cores: 64 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 1048576 l3_cache_size: 23068672 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }



Epoch 00001: val_accuracy improved from -inf to 0.36820, saving model to best_modelcoif6.hdf5
781/781 [==============================] - 106s 128ms/step - loss: 2.5886 - accuracy: 0.3345 - val_loss: 2.2894 - val_accuracy: 0.3682
Epoch 2/50
781/781 [==============================] - ETA: 0s - loss: 1.1396 - accuracy: 0.6593
Epoch 00002: val_accuracy improved from 0.36820 to 0.67924, saving model to best_modelcoif6.hdf5
781/781 [==============================] - 108s 137ms/step - loss: 1.1396 - accuracy: 0.6593 - val_loss: 1.0002 - val_accuracy: 0.6792
Epoch 3/50
781/781 [==============================] - ETA: 0s - loss: 0.7171 - accuracy: 0.7654
Epoch 00003: val_accuracy improved from 0.67924 to 0.76166, saving model to best_modelcoif6.hdf5
781/781 [==============================] - 103s 130ms/step - loss: 0.7171 - accuracy: 0.7654 - val_loss: 0.7177 - val_accuracy: 0.7617
Epoch 4/50
781/781 [==============================] - ETA: 0s - loss: 0.5528 - accuracy: 0.8163
Epoch 00004: val_a

Epoch 28/50
781/781 [==============================] - ETA: 0s - loss: 0.0676 - accuracy: 0.9773
Epoch 00028: val_accuracy improved from 0.93491 to 0.93540, saving model to best_modelcoif6.hdf5
781/781 [==============================] - 104s 131ms/step - loss: 0.0676 - accuracy: 0.9773 - val_loss: 0.2298 - val_accuracy: 0.9354
Epoch 29/50
781/781 [==============================] - ETA: 0s - loss: 0.0703 - accuracy: 0.9760
Epoch 00029: val_accuracy did not improve from 0.93540
781/781 [==============================] - 105s 132ms/step - loss: 0.0703 - accuracy: 0.9760 - val_loss: 0.2954 - val_accuracy: 0.9213
Epoch 30/50
781/781 [==============================] - ETA: 0s - loss: 0.0752 - accuracy: 0.9756
Epoch 00030: val_accuracy did not improve from 0.93540
781/781 [==============================] - 106s 133ms/step - loss: 0.0752 - accuracy: 0.9756 - val_loss: 0.2529 - val_accuracy: 0.9341
Epoch 31/50
781/781 [==============================] - ETA: 0s - loss: 0.0660 - accuracy: 0.9776
